## Project title: Extraction of Lake Area Change through NDWI
* Author: Injal Bhattarai

## Abstract
* The main purpose of the program is to reduce the hefty work and actually calculate the lake area change with precision in python environment. This program uses the NDWI index to calculate the lake area.
* NDWI is used to assess alterations in water body composition. NDWI utilizes green and near infrared bands to emphasize water bodies because water heavily absorbs light in the visible to infrared em spectrum. 
* NDWI can be calculated from the landsat images. Green and Near Infrared bands are used to find the value of each pixel. Formula for NDWI is (Green – NIR)/(Green + NIR)
* Pixel values greater than 2 and less than 1 denote the water content area. 
* External python packages such as geemap, ee, wx along with a few functions such as getfilename and timespan are used in this program. 
* Users have to upload the shapefile of the area they want to calculate the NDWI and input the dates of which they want to find the area of the lake.
* This program calculates the lake area by multiplying the NDWI pixels with the resolution of the ee image. Finally lake area of two different time period can be calculated and lake area change can be discovered. 

## Introduction
* The main purpose of the program is to calculate the total lake area change by reducing the hefty work with precision in python environment. 
* Through this program, any user can actually find the area with water content of the study area. It can be used in micro research such as moisture content in tree canopy to finding river pathways. 
* Calculation of NDWI is a hefty procedure and is generally done in GIS softwares with large number of consecutive procedures. 

* Missing even one of the steps leads to the miscalculation of the whole process hence programming makes it a lot more user friendly and saves a lot of time.
* Packages used in this program: 
    Geemap used in this program is a package for Google Earth Engine (GEE), a cloud computing platform with a multi-petabyte database of satellite images and geographical information. ee package in Python programming enables developers to communicate with Google Earth Engine. WX helps the program to take input from the user. 

## Study Area and Data
* Study area will be defined by the user with the shapefile they upload. In this program, Rara lake of Nepal is used as sample data to find the lake area from 2013 to 2022. Shapefile was created from ArcGIS pro and used in this program. 
* Landsat images will be assessd from Google Earth Engine. 

## Project Design and Processes
1. User inputs the shapefile, study time periods and cloud cover.
2. Program filters out the landsat image accroding to the cloud cover and date.
3. The program uses Landsat8 images if the study period ranges from 2013 to 2022 (recent time) and uses Landsat 5 image if the study period ranges from 1984 to 2011.
4. The NDWI is calculated from the ladsat images where the study area is defined by the user. 
5. The area of the lake is calculated from the produced image where the total area = total number of pixels having value >2 * area of the resolution of the landsat image.
6. Lake area is displayed on top of the ee map as the output of the image.
7. The lake area of both the input years are displayed in sq.m separately and the difference separately. 

* The functions used in this program are:
1. getfilename: This function returns a list of filenames and filepath chosen in the dialog box(the shapefile and its location)
2. time_span: This function returns the study period input by the user, filters the landsat images as input by the user, calculates NDWI, uses visual parameters to display the NDWI image and gives the output as the lake area change in sq.m. 

## Results and Discussion
* The program is mainly dedicated to find the lake area but it can also be used to find out the area of water presence as index NDWI actually gives us the reflection of the water content. Calculation of NDWI in python can save hours of manual work from the GIS and the precision can actually be unparalleled.  

In [1]:
import geemap

In [2]:
import ee
import wx

In [3]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
def getfilename():
    '''Returns a list of filenames and filepath chosen in the dialog box'''
    app = wx.App()
    frame = wx.Frame(None, -1, 'win.py')
    frame.SetDimensions(0,0,200,50)

    openFileDialog = wx.FileDialog(frame, "Open", "", "", 
          "", 
           wx.FD_OPEN | wx.FD_FILE_MUST_EXIST)

    openFileDialog.ShowModal()
    return openFileDialog.GetPath()

In [5]:
collection_l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA'); 
collection_l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_TOA"); 
'''Takes input as satellite images and return image collection'''

outline_shp = getfilename()
outline_ee = geemap.shp_to_ee(outline_shp)
Map.addLayer(outline_ee, {}, "MAP")

Map.centerObject(outline_ee, 13)

In [6]:
def time_span(start, end):
    '''Takes input as start year and end year from the user and returns the area change in float'''
    years = [int(start), int(end)]
    yearList = []
    areaList = []
    cloud_cover = int(input("Enter the preferred cloud cover:\n(Default value 10 will be used if input is not provided)\n") or "10")
    for year in years:
        if int(year) < 2012:
            image = collection_l5
        elif int(year) > 2012 & int(year) < 2023:
            image = collection_l8
        else:
            image = None
        if image == None:
            print ("Enter a valid date")
            return 
        else:
            yearList.append(year)
            collection = ee.ImageCollection(image).filter(ee.Filter.lt('CLOUD_COVER', cloud_cover)).filterBounds(outline_ee)
            select_date = collection.filterDate(str(year)+"-01-01", str(year)+"-12-30")
            clip = select_date.median().clip(outline_ee)

            visArgs = {"min": -0.44, "max": 0.08, "palette": ['ff3f06', 'f0ff06', '08ff26', '0afff4', '0625ff']}
            ndwi =  clip.normalizedDifference(['B3', 'B5']).rename('NDWI')
            '''normalizedDifference is inbuilt geemap function which takes two bands (B3 and B5) input to compute the 
            normalized value'''
            Map.addLayer(ndwi, visArgs, "NDWI_"+str(year))
            ndwi_gt = 0.2;
            water = ndwi.gt(ndwi_gt);
            mask = clip.updateMask(water).addBands(ndwi);
            area = ee.Image.pixelArea();
            lakeArea = water.multiply(area).rename('area')
            imgbands = mask.addBands(lakeArea);
            stats = lakeArea.reduceRegion(reducer=ee.Reducer.sum(), geometry=outline_ee, scale=30)
            area = ee.Number(stats.get('area')).getInfo()
            areaList.append(area)
    
    area_change = areaList[0]-areaList[1]
    print("The lake area in {} year in sq.m. is:".format(yearList[0]), areaList[0])
    print("The lake area in {} year in sq.m. is:".format(yearList[1]), areaList[1])
    print("The change of lake area from {} year to {} year in sq.m. is:".format(yearList[0], yearList[1]), area_change)
    return area_change
        
start_year = input("Enter the first year: ")
end_year = input("Enter the second year: ")
time_span(start_year, end_year)


Enter the first year: 2013
Enter the second year: 2022
Enter the preferred cloud cover:
(Default value 10 will be used if input is not provided)

The lake area in 2013 year in sq.m. is: 10406663.47380443
The lake area in 2022 year in sq.m. is: 10369682.258178711
The change of lake area from 2013 year to 2022 year in sq.m. is: 36981.215625718236


36981.215625718236